# Assignment 1

## Assignment text
1. **[1p]** Download data competition from a Kaggle competition on sentiment prediction from [[https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/data](https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/data)].  Keep only full sentences, i.e. for each `SenteceId` keep only the entry with the lowest `PhraseId`.  Use first 7000 sentences as a `train set` and the remaining 1529 sentences as the `test set`. 

2. **[1p]** Prepare the data for logistic regression:
	Map the sentiment scores $0,1,2,3,4$ to a probability of the sentence being by setting $p(\textrm{positive}) = \textrm{sentiment}/4$.
	Build a dictionary of at most 20000 most frequent words.

3. **[3p]** Treat each document as a bag of words. e.g. if the vocabulary is 
	```
	0: the
	1: good
	2: movie
	3: is
	4: not
	5: a
	6: funny
	```
	Then the encodings can be:
	```
	good:                           [0,1,0,0,0,0,0]
	not good:                       [0,1,0,0,1,0,0] 
	the movie is not a funny movie: [1,0,2,1,1,1,1]
	```
    Train a logistic regression model to predict the sentiment. Compute the correlation between the predicted probabilities and the sentiment. Record the most positive and negative words.
    Please note that in this model each word gets its sentiment parameter $S_w$ and the score for a sentence is 
    $$\text{score}(\text{sentence}) = \sum_{w\text{ in sentence}}S_w$$

4. **[3p]** Now prepare an encoding in which negation flips the sign of the following words. For instance for our vocabulary the encodings become:
	```
	good:                           [0,1,0,0,0,0,0]
	not good:                       [0,-1,0,0,1,0,0]
	not not good:                   [0,1,0,0,0,0,0]
	the movie is not a funny movie: [1,0,0,1,1,-1,-1]
	```
	For best results, you will probably need to construct a list of negative words.
	
	Again train a logistic regression classifier and compare the results to the Bag of Words approach.
	
	Please note that this model still maintains a single parameter for each word, but now the sentence score is
	$$\text{score}(\text{sentence}) = \sum_{w\text{ in sentence}}-1^{\text{count of negations preceeding }w}S_w$$

5. **[5p]** Now also consider emphasizing words such as `very`. They can boost (multiply by a constant >1) the following words.
	Implement learning the modifying multiplier for negation and for emphasis. One way to do this is to introduce a model which has:
	- two modifiers, $N$ for negation and $E$ for emphasis
	- a sentiment score $S_w$ for each word 
And score each sentence as:
$$\text{score}(\text{sentence}) = \sum_{w\text{ in sentence}}N^{\text{\#negs prec. }w}E^{\text{\#emphs prec. }w}S_w$$

You will need to implement a custom logistic regression model to support it.

6. **[2pb]** Propose, implement, and evaluate an extension to the above model.


In [209]:
# imports
import numpy as np
import pandas as pd
from collections import defaultdict 
import re

from sklearn.linear_model import LogisticRegression
import scipy.optimize as sopt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from mord import LogisticAT

## Task 1.

In [2]:
def regex(text):
    text = re.sub(r'[^\w\s]', '', text.lower())
    return text

In [3]:
df = pd.read_csv('train.tsv', sep='\t')
# test_df = pd.read_csv('test.tsv', sep='\t')

df = df.groupby(['SentenceId'], 
                          as_index=False).agg({'PhraseId' : 'min',
                                               'Phrase' : 'first',
                                               'Sentiment' : 'first'})

df['Sentiment'] = df['Sentiment'].apply(lambda x: x / 4)
df = df.drop(['PhraseId', 'SentenceId'], axis=1)
df.Phrase = df.Phrase.apply(lambda row: regex(row))
df.shape

(8529, 2)

In [4]:
df.head()

,Phrase,Sentiment
0,a series of escapades demonstrating the adage ...,0.25
1,this quiet introspective and entertaining ind...,1.00
2,even fans of ismail merchant s work i suspect...,0.25
3,a positively thrilling combination of ethnogra...,0.75
4,aggressive selfglorification and a manipulativ...,0.25


In [5]:
# target value counts
df['Sentiment'].value_counts()

0.75    2321
0.25    2200
0.50    1655
1.00    1281
0.00    1072
Name: Sentiment, dtype: int64

In [6]:
# test and train split
train_df = df.iloc[: 7000]
test_df = df.iloc[7000: ]

print(train_df.shape, test_df.shape)

(7000, 2) (1529, 2)


# Task 2 & 3

In [119]:
class Logistic_Regression:
    def __init__(self, max_iter=500, solver_calls=5,lambda_ = 0.1,Theta=None,
                 solver=sopt.fmin_l_bfgs_b, debug=False):
        self.Theta = Theta
        self.solver_calls = solver_calls
        self.max_iter = max_iter
        self.solver = solver
        self.debug = debug
        self.lambda_ = lambda_
    
    def __sigmoid(self, x):
        return 1 / (1 + np.exp(-x))    
    
    def __logreg_loss(self, Theta, X, Y):
        Theta = Theta.astype(np.float64)
        X = X.astype(np.float64)
        Y = Y.astype(np.float64)
        
        Z = np.dot(Theta, X.T)

        SZ = self.__sigmoid(Z)
        Y_ = Y[:,np.newaxis]
        nll = -np.sum((Y_ * np.log2(SZ + 1e-50) + (1-Y_) * np.log2(1 - SZ + 1e-50)))

        nll += (self.lambda_/2) * np.sum(Theta**2)

        grad = np.dot(X.T, (SZ - Y).T )
        grad = grad.reshape(Theta.shape) + self.lambda_ * Theta

        return nll, grad
    
    def fit(self, X, y):
        Theta = self.Theta
        if Theta is None:
            Theta = np.ones(X.shape[1]+1)
        
        X_with_ones = np.hstack((np.ones((X.shape[0], 1)), X))
      
        for i in tqdm(range(self.solver_calls), desc='Calculating Theta', position=0):
            Theta = self.solver(lambda th: self.__logreg_loss(th, X_with_ones, y), 
                                Theta, maxiter=self.max_iter)[0]
        self.Theta = Theta
    
    def predict(self, X):
        X_with_ones = np.hstack((np.ones((X.shape[0], 1)), X))
        preds = np.dot(self.Theta, X_with_ones.T)
#         preds = np.dot(self.Theta, X_with_ones.T)
        return preds, self.__sigmoid(preds)

In [101]:
class MyCountVectorizer:
    def __init__(self, min_df=-1, max_df=1e18, binary=False):
        self.min_df = min_df
        self.max_df = max_df
        self.binary = binary
    
    def fit(self, df):
        words_cnt = defaultdict(int)
        col = df.columns[0]
        
        for i in range(len(df)):
            text = df.iloc[i][col]
            for word in text.split():
                words_cnt[word] += 1
                
        all_words = []
        for word, cnt in words_cnt.items():
            if self.min_df <= cnt <= self.max_df:
                all_words.append(word)
                
        self.all_words_ids = {w:i for i,w in enumerate(all_words)}
        self.width = len(all_words)
        
    
    def transform(self, df):
        col = df.columns[0]
        count_matrix = np.zeros([len(df), self.width], \
                                dtype=np.int32)
        
        for i in range(len(df)):
            text = df.iloc[i][col]
            words_cnt = defaultdict(int)
            
            for word in text.split():
                words_cnt[word] += 1
            
            for word, cnt in words_cnt.items():
                if word in self.all_words_ids:
                    pos = self.all_words_ids[word]
                    if self.binary:
                        count_matrix[i][pos] = 1
                    else:
                        count_matrix[i][pos] = cnt
                    
        return count_matrix

In [54]:
# takes LR output and maps for closest targets

targets = np.array([0, 0.25, 0.5, 0.75, 1])
def find_nearest(value, array=targets):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [18]:
# Data embedding 

cv = MyCountVectorizer()
cv.fit(train_df)

X_train = cv.transform(train_df) 
X_test = cv.transform(test_df)

y_train = train_df.Sentiment
y_test = test_df.Sentiment

## My Logistic Regression

In [170]:
%%time

my_LR = Logistic_Regression(max_iter=1000, lambda_=10, Theta=None, debug=False, solver_calls=1)
my_LR.fit(X_train, y_train)

Calculating Theta: 100%|████████████████████████████████████████████████████████████████| 1/1 [02:18<00:00, 138.71s/it]


Wall time: 2min 19s


In [171]:
train_preds = my_LR.predict(X_train)
test_preds = my_LR.predict(X_test)

test_preds = list(map(lambda x: find_nearest(x), test_preds[1]))
train_preds = list(map(lambda x: find_nearest(x), train_preds[1]))

In [172]:
print(f'Train acc: {(np.array(train_preds) == np.array(y_train)).mean() * 100:.3f}%')
print(f'Test acc: {(np.array(test_preds) == np.array(y_test)).mean() * 100:.3f}%')

Train acc: 35.243%
Test acc: 29.562%


In [205]:
print('Train')
print(pd.Series(train_preds).value_counts())

print('\nActual Train')
print(pd.Series(y_test).value_counts())

Train
0.50    4236
0.75    1647
0.25    1088
1.00      18
0.00      11
dtype: int64

Actual Train
0.75    419
0.25    390
0.50    293
1.00    233
0.00    194
Name: Sentiment, dtype: int64


In [198]:
print('Test')
print(pd.Series(test_preds).value_counts())

print('\nActual Test')
print(pd.Series(y_test).value_counts())

Train
0.50    1026
0.75     295
0.25     201
0.00       5
1.00       2
dtype: int64

Actual train
0.75    419
0.25    390
0.50    293
1.00    233
0.00    194
Name: Sentiment, dtype: int64


## Sklearn

In [199]:
%%time

LR = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
LR.fit(X_train, y_train * 4)

Wall time: 35.5 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [202]:
train_preds2 = LR.predict(X_train)
test_preds2 = LR.predict(X_test)

print(f'Train acc: {(np.array(train_preds2) == np.array(y_train * 4)).mean() * 100:.3f}%')
print(f'Test acc: {(np.array(test_preds2) == np.array(y_test * 4)).mean() * 100:.3f}%')

Train acc: 96.700%
Test acc: 39.111%


In [207]:
print('Train')
print(pd.Series(train_preds2).value_counts())

print('\nActual Train')
print(pd.Series(y_test).value_counts())

Train
3.0    1913
1.0    1866
2.0    1355
4.0    1018
0.0     848
dtype: int64

Actual Train
0.75    419
0.25    390
0.50    293
1.00    233
0.00    194
Name: Sentiment, dtype: int64


In [206]:
print('Test')
print(pd.Series(test_preds2).value_counts())

print('\nActual Test')
print(pd.Series(y_test).value_counts())

Test
3.0    459
1.0    432
2.0    292
4.0    202
0.0    144
dtype: int64

Actual Test
0.75    419
0.25    390
0.50    293
1.00    233
0.00    194
Name: Sentiment, dtype: int64


## Oridinal Logistic Regression

In [ ]:
%%time
oridinal_LR = LogisticAT(alpha=0)
oridinal_LR.fit(X_train, (y_train * 4).astype(np.int))

In [11]:
%%time
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
cv.fit(train_df.Phrase)

X_train = cv.transform(train_df) 
X_test = cv.transform(test_df)


param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, train_df.Sentiment * 4)

# print("Best cross-validation score: {:.2f}".format(grid.best_score_))
# print("Best parameters: ", grid.best_params_)
# print("Best estimator: ", grid.best_estimator_)

ValueError: Found input variables with inconsistent numbers of samples: [2, 7000]

In [12]:
import matplotlib.pyplot as plt
import mglearn

feature_names = cv.get_feature_names()
mglearn.tools.visualize_coefficients(grid.best_estimator_.coef_, feature_names, n_top_features=25)
plt.show()

C:\Users\Febrin\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
C:\Users\Febrin\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'